In [1]:
from pydes.process import *
import pandas as pd

/var/folders/cr/hy92hn010tqdqdlyhxvz2fmw0000gn/T/ipykernel_3769/993140321.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
class State(Component):
    def __init__(self, value):
        self.value = value


class Process1(Component):
    def __init__(self, sim: Simulator):
        self.sim = sim

    def main(self):
        for _ in range(10):
            self.sim.record(
                self,
                "waiting",
                1,
            )
            self.sim.sleep(10)
            self.sim.record(self, "waiting", 0)


class Process2(Component):
    def __init__(self, sim: Simulator, state: State):
        self.sim = sim
        self.state = state

    def main(self):
        for _ in range(10):
            self.sim.record(self, "waiting", 1)
            self.sim.sleep(2)
            self.sim.record(self, "waiting", 0)

        self.sim.record(state, "change state", True)
        self.state.value = True


class Process3(Component):
    def __init__(self, sim: Simulator, state: State):
        self.sim = sim
        self.state = state

    def main(self):
        self.sim.record(self, "wait state", 1)
        self.sim.wait_for(lambda: self.state.value)
        self.sim.record(self, "wait state", 0)

In [3]:
sim = Simulator(trace=False)
state = State(value=False)
fb1 = Process1(sim)
fb1_1 = Process1(sim)
fb2 = Process2(sim, state)
fb3 = Process3(sim, state)

sim.activate(fb1)
sim.activate(fb1_1, at=1)
sim.activate(fb2, at=20)
sim.activate(fb3)
sim.simulate()

In [4]:
records = pd.DataFrame(sim.records())
records

,time,component,description,value
0,0,Process1.0,waiting,1
1,0,Process3.0,wait state,1
2,1,Process1.1,waiting,1
3,10,Process1.0,waiting,0
4,10,Process1.0,waiting,1
...,...,...,...,...
58,90,Process1.0,waiting,1
59,91,Process1.1,waiting,0
60,91,Process1.1,waiting,1
61,100,Process1.0,waiting,0
